# Introduction

This is a script to show descriptive statistics of blood glucose values from a [Nightscout](https://nightscout.github.io/) instance. It simply displays blood glucose summary statistics stratified for each day the pump site has been in use.

# Usage

1. Fill the form with Nightscout API credentials and choose time range. Note that the Nightscout URL should be the bare base url to the Nightscout instance. 
1. From the menu bar select Runtime: Run All

Alternatively, download the Notebook file [here](https://github.com/mzsoftworks/nightscout-site-age) and run the program on your local computer.

In [ ]:
# @title Configuration { display-mode: "form" }

API_SECRET = "" # @param {type:"string"}
NS_BASE_URL = "" # @param {type:"string"}
START_DATE = "2023-08-01" # @param {type:"date"}
END_DATE = "2023-09-01" # @param {type:"date"}
BG_UNIT = "mg/dl" # @param ["mg/dl", "mmol/l"]

In [ ]:
# @title API requests { display-mode: "form" }

from hashlib import sha1
from datetime import date, datetime, timedelta
import statistics
from concurrent.futures import ThreadPoolExecutor

import requests
import pandas as pd
import seaborn as sns
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt

pd.set_option("display.precision", 1)
    
days_to_iterate = (date.fromisoformat(END_DATE) - date.fromisoformat(START_DATE)).days+1

print(f"Loading {days_to_iterate} days...")

if days_to_iterate < 1:
    raise RuntimeError("No days to fetch")

credential_headers = {"Api-Secret": sha1(bytes(API_SECRET,"utf8")).hexdigest()}

def fetch_day(day_offset):
    date_lt = pd.to_datetime("now",utc=True) - pd.Timedelta(days=day_offset)
    date_gte = date_lt - pd.Timedelta(days=1)
    url = f"{NS_BASE_URL}/api/v1/entries.json?find[date][$gte]={int(date_gte.timestamp()*1000)}&find[date][$lt]={int(date_lt.timestamp()*1000)}&count=10000"
    response = requests.get(url, headers=credential_headers)
    response.raise_for_status()
    return pd.read_json(response.text,convert_dates=["dateString"])

with ThreadPoolExecutor(max_workers=10) as pool:
    days_of_entries = pool.map(fetch_day, range(days_to_iterate))

entries = pd.concat(days_of_entries)

url = f"{NS_BASE_URL}/api/v1/treatments.json?find%5BeventType%5D=Site%20Change&find%5Bcreated_at%5D%5B%24lt%5D=2050-10-04"
res = requests.get(url,headers=credential_headers)
res.raise_for_status()
changes = pd.read_json(res.text, convert_dates=["created_at"])

In [ ]:
# @title Descriptive statistics { display-mode: "form" }

def tir(glucoses):
    return np.sum((glucoses >= 3.9/0.0555) & (glucoses < 10/0.0555)) / len(glucoses) * 100

def low(glucoses):
    return np.sum(glucoses < 3.9/0.0555) / len(glucoses) * 100

def high(glucoses):
    return np.sum(glucoses >= 10/0.0555) / len(glucoses) * 100

entries_sort = entries.sort_values("dateString")
changes_sort = changes.sort_values("created_at")

merged_df = pd.merge_asof(entries_sort, changes_sort, left_on='dateString', right_on='created_at')

merged_df["site_age_days"] = (merged_df["dateString"] - merged_df["created_at"]).dt.floor("D").dt.days
merged_df["mmol/l"] = merged_df["sgv"] * 0.0555
merged_df["mg/dl"] = merged_df["sgv"]

merged_df.groupby("site_age_days").agg({
    "mg/dl" if BG_UNIT == "mg/dl" else "mmol/l":["count", "mean","median","std"],
    "sgv": [tir,low,high]
})

In [ ]:
# @title Boxplot { display-mode: "form" }

sns.boxplot(data=merged_df, y="mg/dl" if BG_UNIT =="mg/dl" else "mmol/l",x="site_age_days",orient="v");